In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import os

#Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score,classification_report

from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /Users/priya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
messages = pd.read_csv("/Users/priya/Desktop/MYLAPIDOCS/Tech/KrishNaik/SpamMailDetection/spam_ham_dataset.csv")
messages.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [3]:
df = messages.iloc[:,2:]
df.head()

,text,label_num
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\r\nthis deal is t...,0


In [4]:
df.isna().sum()

text         0
label_num    0
dtype: int64

In [5]:
df.label_num.value_counts()

0    3672
1    1499
Name: label_num, dtype: int64

so ham = 3672
spam = 1499

In [6]:
total_stopwords = set([word.replace("'",'') for word in stopwords.words('english')])

def preprocess_text(text):
    #text = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    #text = text.lower()
    text = re.sub('[^a-zA-Z]',' ',str(text))
    text = text.replace("'",'')
    text = text.lower()
    words = text.split()
    words = [lemma.lemmatize(word) for word in words if (word not in total_stopwords) and (len(word)>1)] # Remove stop words
    text = " ".join(words)
    return text

In [7]:
X = df['text'].apply(preprocess_text)
X.head()

0    subject enron methanol meter follow note gave ...
1    subject hpl nom january see attached file hpln...
2    subject neon retreat ho ho ho around wonderful...
3    subject photoshop window office cheap main tre...
4    subject indian spring deal book teco pvr reven...
Name: text, dtype: object

In [8]:
X.shape

(5171,)

In [9]:
y = df['label_num']
y.head()

0    0
1    0
2    0
3    1
4    0
Name: label_num, dtype: int64

In [10]:
# Creating the TF-IDF model
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
X = tf.fit_transform(X).toarray()

In [11]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
#test-train split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.20, stratify=y)

In [13]:
#logistic regression
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(x_train,y_train)

#predicting train set results
y_trainpred = lr.predict(x_train)
# Predicting the Test set results
y_pred = lr.predict(x_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
clf_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True))
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
score = accuracy_score(y_train,y_trainpred)
score1 = accuracy_score(y_test,y_pred)
score2 = precision_score(y_test,y_pred,average='micro')
score3= recall_score(y_test,y_pred,average='micro')
print("\n")
print("Training Accuracy is ",round(score*100,2),"%")
print("Testing Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))
print(f"CLASSIFICATION REPORT:\n{clf_report}")

Confusion Matrix:
 [[719  16]
 [  1 299]]


Training Accuracy is  99.52 %
Testing Accuracy is  98.36 %
Precision is  0.98
Recall is  0.98
CLASSIFICATION REPORT:
                    0           1  accuracy    macro avg  weighted avg
precision    0.998611    0.949206  0.983575     0.973909      0.984291
recall       0.978231    0.996667  0.983575     0.987449      0.983575
f1-score     0.988316    0.972358  0.983575     0.980337      0.983691
support    735.000000  300.000000  0.983575  1035.000000   1035.000000


In [14]:
#SVM model
from sklearn.svm import SVC
svm= SVC(kernel='linear')
svm.fit(x_train,y_train)


#predicting train set results
y_trainpred = svm.predict(x_train)

# Predicting the Test set results
y_pred = svm.predict(x_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
clf_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True))
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
score = accuracy_score(y_train,y_trainpred)
score1 = accuracy_score(y_test,y_pred)
score2 = precision_score(y_test,y_pred,average='micro')
score3= recall_score(y_test,y_pred,average='micro')
print("\n")
print("Training Accuracy is ",round(score*100,2),"%")
print("Testing Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))
print(f"CLASSIFICATION REPORT:\n{clf_report}")

Confusion Matrix:
 [[725  10]
 [  3 297]]


Training Accuracy is  99.93 %
Testing Accuracy is  98.74 %
Precision is  0.99
Recall is  0.99
CLASSIFICATION REPORT:
                    0           1  accuracy    macro avg  weighted avg
precision    0.995879    0.967427   0.98744     0.981653      0.987632
recall       0.986395    0.990000   0.98744     0.988197      0.987440
f1-score     0.991114    0.978583   0.98744     0.984849      0.987482
support    735.000000  300.000000   0.98744  1035.000000   1035.000000


In [15]:
# Bernoulli NB

# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import BernoulliNB
classifier = BernoulliNB(alpha=0.8)
classifier.fit(x_train, y_train)


#predicting train set results
y_trainpred = classifier.predict(x_train)

# Predicting the Test set results
y_pred = classifier.predict(x_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
clf_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True))
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
score = accuracy_score(y_train,y_trainpred)
score1 = accuracy_score(y_test,y_pred)
score2 = precision_score(y_test,y_pred,average='micro')
score3= recall_score(y_test,y_pred,average='micro')
print("\n")
print("Training Accuracy is ",round(score*100,2),"%")
print("Testing Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))
print(f"CLASSIFICATION REPORT:\n{clf_report}")

Confusion Matrix:
 [[718  17]
 [124 176]]


Training Accuracy is  88.32 %
Testing Accuracy is  86.38 %
Precision is  0.86
Recall is  0.86
CLASSIFICATION REPORT:
                    0           1  accuracy    macro avg  weighted avg
precision    0.852732    0.911917  0.863768     0.882324      0.869887
recall       0.976871    0.586667  0.863768     0.781769      0.863768
f1-score     0.910590    0.713996  0.863768     0.812293      0.853606
support    735.000000  300.000000  0.863768  1035.000000   1035.000000
